# 05. Model Visualization and Feature Space Analysis

This notebook creates advanced visualizations to understand food relationships in feature space and analyze model behavior through dimensionality reduction techniques.

## Objectives:

- Visualize food relationships using PCA and t-SNE
- Analyze feature importance and model interpretability
- Create prediction confidence and error analysis
- Understand food clustering patterns in nutritional space

**Prerequisites**: Run notebooks 01-04 first.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from itertools import cycle

# Set random seed for reproducibility
np.random.seed(10)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
print("🎨 Visualization libraries imported successfully!")

In [ ]:
# Load model data and results
print("📂 Loading Model Data and Analysis Results")
print("=" * 45)

try:
    # Load data from previous notebooks
    X_scaled = joblib.load('../models/X_scaled.pkl')
    food_lookup = joblib.load('../models/food_lookup.pkl')
    eval_data = joblib.load('../models/eval_subset.pkl')
    final_model = joblib.load('../models/optimized_similarity_model.pkl')
    model_config = joblib.load('../models/model_config.pkl')
    similarity_results = joblib.load('../models/similarity_analysis_results.pkl')
    
    X_eval = eval_data['X_eval']
    food_eval = eval_data['food_eval']
    best_params = model_config['best_params']
    feature_columns = model_config['feature_columns']
    
    print(f"✅ Loaded data: {X_scaled.shape} samples, {len(feature_columns)} features")
    print(f"✅ Evaluation subset: {len(X_eval)} samples")
    print(f"✅ Food categories: {len(food_lookup['category'].unique())} categories")
    
except FileNotFoundError as e:
    print(f"❌ Error loading data: {e}")
    print("Please run the previous notebooks first (01-04)")
    raise

except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

In [ ]:
# 1. Food Nutritional Space Visualization
print("\n1️⃣ FOOD NUTRITIONAL SPACE VISUALIZATION")
print("-" * 40)

# Create sample for visualization (subset for performance)
sample_size = min(1000, len(X_scaled))
sample_indices = np.random.choice(len(X_scaled), sample_size, replace=False)
X_sample = X_scaled[sample_indices]
food_sample = food_lookup.iloc[sample_indices].reset_index(drop=True)

# Apply dimensionality reduction techniques
print("Applying dimensionality reduction...")

# PCA
pca_viz = PCA(n_components=2)
X_pca = pca_viz.fit_transform(X_sample)

# t-SNE  
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_sample)

# Create category color mapping
unique_categories = food_sample['category'].unique()
colors = plt.cm.Set3(np.linspace(0, 1, len(unique_categories)))
color_map = dict(zip(unique_categories, colors))
category_colors = [color_map[cat] for cat in food_sample['category']]

# Create label encoder for numerical representation
label_encoder = LabelEncoder()
y_sample = label_encoder.fit_transform(food_sample['category'])

# Get predictions from final model
best_model_pred = final_model.predict(X_sample)

# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# PCA - Food categories
scatter1 = axes[0,0].scatter(X_pca[:, 0], X_pca[:, 1], c=category_colors, alpha=0.6)
axes[0,0].set_title('PCA - Food Categories in Nutritional Space')
axes[0,0].set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} variance)')
axes[0,0].set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} variance)')

# Add legend for categories (show first 10 for readability)
legend_categories = unique_categories[:10]
legend_colors = [color_map[cat] for cat in legend_categories]
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                             markerfacecolor=color, markersize=8, label=cat) 
                  for cat, color in zip(legend_categories, legend_colors)]
axes[0,0].legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.3, 1))

# PCA - Best Model Predictions
scatter2 = axes[0,1].scatter(X_pca[:, 0], X_pca[:, 1], c=best_model_pred, cmap='tab10', alpha=0.6)
axes[0,1].set_title('PCA - Model Predictions')
axes[0,1].set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} variance)')
axes[0,1].set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter2, ax=axes[0,1])

# t-SNE - True labels
scatter3 = axes[1,0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_sample, cmap='tab10', alpha=0.6)
axes[1,0].set_title('t-SNE - True Food Categories')
axes[1,0].set_xlabel('t-SNE Component 1')
axes[1,0].set_ylabel('t-SNE Component 2')
plt.colorbar(scatter3, ax=axes[1,0])

# t-SNE - Best Model Predictions
scatter4 = axes[1,1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=best_model_pred, cmap='tab10', alpha=0.6)
axes[1,1].set_title('t-SNE - Model Predictions')
axes[1,1].set_xlabel('t-SNE Component 1')
axes[1,1].set_ylabel('t-SNE Component 2')
plt.colorbar(scatter4, ax=axes[1,1])

plt.tight_layout()
plt.show()

print(f"\n✅ Feature space visualization completed!")
print(f"📊 PCA explains {pca_viz.explained_variance_ratio_.sum():.2%} of total variance")
print(f"📊 Visualized {sample_size} samples from {len(unique_categories)} food categories")

## Why Feature Space Visualization?

**Purpose**: Understand how food items cluster in the feature space and evaluate model decision boundaries.

**Why This Matters**:

- **Data Distribution**: Visualize how different food categories are distributed in nutritional space
- **Model Behavior**: See how KNN models make decisions based on local neighborhoods
- **Clustering Patterns**: Identify natural groupings of similar foods
- **Decision Boundary Analysis**: Compare true labels vs model predictions visually

**Techniques Used**:

- **PCA (Principal Component Analysis)**: Linear dimensionality reduction preserving maximum variance
  - Good for understanding overall data structure
  - Shows which nutritional combinations explain most variation
- **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: Non-linear reduction preserving local structure
  - Better for visualizing clusters and local neighborhoods
  - Reveals hidden patterns in high-dimensional nutritional data

**What We Learn**:

- Whether food categories form distinct clusters
- How well KNN models capture these natural groupings
- Potential misclassification patterns
- Data quality issues (outliers, overlapping categories)

**Business Value**: Understanding food similarity patterns helps improve meal planning algorithms and identify opportunities for better categorization.


In [ ]:
# 2. Model Prediction Confidence and Error Analysis
print("\n2️⃣ PREDICTION CONFIDENCE & ERROR ANALYSIS")
print("-" * 45)

# Create a test set for confidence analysis using evaluation data
X_test_scaled = X_eval.values
y_test = label_encoder.fit_transform(food_eval['category'])

# Get prediction probabilities from best model
best_model_proba = final_model.predict_proba(X_test_scaled)

# Calculate prediction confidence (max probability)
conf = np.max(best_model_proba, axis=1)

# Identify correct and incorrect predictions
correct = (final_model.predict(X_test_scaled) == y_test)

# Create confidence analysis visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Confidence distribution for correct vs incorrect predictions
axes[0,0].hist(conf[correct], bins=30, alpha=0.7, label='Correct', color='green')
axes[0,0].hist(conf[~correct], bins=30, alpha=0.7, label='Incorrect', color='red')
axes[0,0].set_title('Prediction Confidence Distribution')
axes[0,0].set_xlabel('Confidence (Max Probability)')
axes[0,0].set_ylabel('Frequency')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Confidence vs Accuracy relationship (calibration curve)
confidence_bins = np.linspace(0, 1, 11)
bin_centers = (confidence_bins[:-1] + confidence_bins[1:]) / 2

accuracies = []
for i in range(len(confidence_bins)-1):
    mask = (conf >= confidence_bins[i]) & (conf < confidence_bins[i+1])
    if np.sum(mask) > 0:
        accuracies.append(np.mean(correct[mask]))
    else:
        accuracies.append(0)

axes[0,1].plot(bin_centers, accuracies, 'o-', label='Model Calibration', color='blue')
axes[0,1].plot([0, 1], [0, 1], '--', color='gray', alpha=0.7, label='Perfect Calibration')
axes[0,1].set_title('Model Calibration (Confidence vs Accuracy)')
axes[0,1].set_xlabel('Prediction Confidence')
axes[0,1].set_ylabel('Actual Accuracy')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Error analysis by food category
error_analysis = pd.DataFrame({
    'Category': label_encoder.classes_,
    'Errors': [np.sum((y_test == i) & ~correct) for i in range(len(label_encoder.classes_))],
    'Total_Samples': [np.sum(y_test == i) for i in range(len(label_encoder.classes_))]
})

error_analysis['Error_Rate'] = error_analysis['Errors'] / error_analysis['Total_Samples']
error_analysis = error_analysis.sort_values('Error_Rate', ascending=True)

# Show top 15 categories with most errors
top_error_categories = error_analysis.tail(15)
axes[1,0].barh(range(len(top_error_categories)), top_error_categories['Error_Rate'], 
               color='lightcoral', alpha=0.8)
axes[1,0].set_yticks(range(len(top_error_categories)))
axes[1,0].set_yticklabels(top_error_categories['Category'], fontsize=9)
axes[1,0].set_title('Error Rate by Food Category (Top 15)')
axes[1,0].set_xlabel('Error Rate')
axes[1,0].grid(True, alpha=0.3)

# Confidence statistics by category
conf_stats = pd.DataFrame({
    'Category': label_encoder.classes_,
    'Avg_Confidence': [np.mean(conf[y_test == i]) if np.sum(y_test == i) > 0 else 0 for i in range(len(label_encoder.classes_))],
    'Std_Confidence': [np.std(conf[y_test == i]) if np.sum(y_test == i) > 0 else 0 for i in range(len(label_encoder.classes_))]
})

conf_stats = conf_stats.sort_values('Avg_Confidence', ascending=False).head(15)

axes[1,1].barh(range(len(conf_stats)), conf_stats['Avg_Confidence'], 
               xerr=conf_stats['Std_Confidence'], capsize=3,
               color='skyblue', alpha=0.8)
axes[1,1].set_yticks(range(len(conf_stats)))
axes[1,1].set_yticklabels(conf_stats['Category'], fontsize=9)
axes[1,1].set_title('Average Confidence by Food Category (Top 15)')
axes[1,1].set_xlabel('Average Confidence')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Average Prediction Confidence: {np.mean(conf):.3f}")
print(f"📊 High Confidence Predictions (>0.8): {np.sum(conf > 0.8)}/{len(conf)} ({np.mean(conf > 0.8):.1%})")
print(f"📊 Low Confidence Predictions (<0.5): {np.sum(conf < 0.5)}/{len(conf)} ({np.mean(conf < 0.5):.1%})")

# Show categories with highest error rates
worst_categories = error_analysis.nlargest(3, 'Error_Rate')[['Category', 'Error_Rate', 'Total_Samples']]
print(f"\n⚠️ Categories with Highest Error Rates:")
for _, row in worst_categories.iterrows():
    print(f"   {row['Category']}: {row['Error_Rate']:.2%} ({row['Total_Samples']} samples)")

## Why Prediction Confidence & Error Analysis?

**Purpose**: Evaluate model reliability and understand prediction uncertainty for practical deployment.

**Why This Matters**:

- **Confidence Calibration**: Know when the model is uncertain vs confident
- **Error Pattern Analysis**: Identify systematic weaknesses in model predictions
- **Category-Specific Performance**: Some food categories may be harder to classify
- **Production Reliability**: Understand when to trust model predictions

**What We Analyze**:

- **Confidence Distribution**: How often is the model confident vs uncertain?
- **Calibration Curves**: Does high confidence actually mean high accuracy?
- **Error by Category**: Which food types cause the most classification errors?
- **Confidence Thresholds**: What confidence level should trigger manual review?

**Key Insights**:

- **Well-calibrated models**: High confidence should correlate with high accuracy
- **Error patterns**: Reveal data quality issues or feature limitations
- **Threshold setting**: Balance automation vs manual oversight

**Business Value**:

- Set appropriate confidence thresholds for automated meal planning
- Identify categories needing human review
- Improve user trust through transparent uncertainty communication
- Optimize the balance between automation and accuracy


In [ ]:
# 3. Feature Importance and Model Interpretability
print("\n3️⃣ FEATURE IMPORTANCE & MODEL INTERPRETABILITY")
print("-" * 50)

# Since KNN doesn't have built-in feature importance, we'll use permutation importance
print("Calculating permutation importance (this may take a moment...)")

# Calculate permutation importance for best model
perm_importance = permutation_importance(
    final_model, X_test_scaled, y_test, 
    n_repeats=10, random_state=42, scoring='f1_macro'
)

# Create feature importance dataframe
feature_importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=True)

# Create feature importance visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Feature importance bar plot
y_pos = np.arange(len(feature_importance_df))
axes[0].barh(y_pos, feature_importance_df['Importance'], 
             xerr=feature_importance_df['Std'],
             color='skyblue', alpha=0.8)
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(feature_importance_df['Feature'])
axes[0].set_xlabel('Permutation Importance')
axes[0].set_title('Feature Importance - Optimized Model')
axes[0].grid(True, alpha=0.3)

# Feature correlation with prediction errors
print("\n🔍 Analyzing feature correlation with prediction errors...")

# Calculate feature statistics for correct vs incorrect predictions
feature_analysis = pd.DataFrame()
for i, feature in enumerate(feature_columns):
    feature_values = X_test_scaled[:, i]
    
    feature_analysis = pd.concat([feature_analysis, pd.DataFrame({
        'Feature': [feature],
        'Correct_Mean': [np.mean(feature_values[correct])],
        'Incorrect_Mean': [np.mean(feature_values[~correct])],
        'Difference': [abs(np.mean(feature_values[correct]) - np.mean(feature_values[~correct]))]
    })], ignore_index=True)

feature_analysis = feature_analysis.sort_values('Difference', ascending=True)

# Plot feature difference analysis
y_pos2 = np.arange(len(feature_analysis))
axes[1].barh(y_pos2, feature_analysis['Difference'], 
             color='lightcoral', alpha=0.8)
axes[1].set_yticks(y_pos2)
axes[1].set_yticklabels(feature_analysis['Feature'])
axes[1].set_xlabel('Mean Difference (Correct vs Incorrect)')
axes[1].set_title('Feature Value Differences in Errors')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Top 3 Most Important Features:")
top_features = feature_importance_df.nlargest(3, 'Importance')
for _, row in top_features.iterrows():
    print(f"   {row['Feature']}: {row['Importance']:.4f} ± {row['Std']:.4f}")

print("\n📊 Features with Largest Error Correlation:")
top_error_features = feature_analysis.nlargest(3, 'Difference')
for _, row in top_error_features.iterrows():
    print(f"   {row['Feature']}: {row['Difference']:.4f} difference")

# Save feature importance results
feature_results = {
    'feature_importance': feature_importance_df.to_dict(),
    'error_correlation': feature_analysis.to_dict(),
    'top_features': top_features['Feature'].tolist()
}
joblib.dump(feature_results, '../models/feature_analysis.pkl')
print("\n✅ Saved feature importance analysis")

In [ ]:
# 4. Advanced Food Cluster Analysis
print("\n4️⃣ ADVANCED FOOD CLUSTER ANALYSIS")
print("-" * 35)

# Perform detailed cluster analysis using both PCA and t-SNE results
print("🔍 Analyzing food clusters in reduced dimensional space...")

# Analyze clusters in PCA space
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Determine optimal number of clusters for PCA space
silhouette_scores_pca = []
K_range = range(3, 12)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_pca)
    silhouette_avg = silhouette_score(X_pca, cluster_labels)
    silhouette_scores_pca.append(silhouette_avg)

optimal_k_pca = K_range[np.argmax(silhouette_scores_pca)]
print(f"📊 Optimal clusters for PCA space: {optimal_k_pca}")

# Perform clustering with optimal K
kmeans_pca = KMeans(n_clusters=optimal_k_pca, random_state=42, n_init=10)
pca_clusters = kmeans_pca.fit_predict(X_pca)

# Similar analysis for t-SNE space
silhouette_scores_tsne = []
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X_tsne)
    silhouette_avg = silhouette_score(X_tsne, cluster_labels)
    silhouette_scores_tsne.append(silhouette_avg)

optimal_k_tsne = K_range[np.argmax(silhouette_scores_tsne)]
print(f"📊 Optimal clusters for t-SNE space: {optimal_k_tsne}")

kmeans_tsne = KMeans(n_clusters=optimal_k_tsne, random_state=42, n_init=10)
tsne_clusters = kmeans_tsne.fit_predict(X_tsne)

# Create comprehensive cluster visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. PCA Clusters
scatter1 = axes[0,0].scatter(X_pca[:, 0], X_pca[:, 1], c=pca_clusters, cmap='tab10', alpha=0.6)
axes[0,0].set_title(f'PCA Clusters (K={optimal_k_pca})')
axes[0,0].set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} variance)')
axes[0,0].set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} variance)')
plt.colorbar(scatter1, ax=axes[0,0])

# 2. t-SNE Clusters
scatter2 = axes[0,1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=tsne_clusters, cmap='tab10', alpha=0.6)
axes[0,1].set_title(f't-SNE Clusters (K={optimal_k_tsne})')
axes[0,1].set_xlabel('t-SNE Component 1')
axes[0,1].set_ylabel('t-SNE Component 2')
plt.colorbar(scatter2, ax=axes[0,1])

# 3. Silhouette Score Comparison
axes[0,2].plot(K_range, silhouette_scores_pca, 'o-', label='PCA', color='blue')
axes[0,2].plot(K_range, silhouette_scores_tsne, 'o-', label='t-SNE', color='red')
axes[0,2].set_xlabel('Number of Clusters')
axes[0,2].set_ylabel('Silhouette Score')
axes[0,2].set_title('Cluster Quality vs Number of Clusters')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# 4. Cluster composition analysis
pca_cluster_composition = pd.DataFrame({
    'Category': food_sample['category'],
    'PCA_Cluster': pca_clusters
})

cluster_category_counts = pca_cluster_composition.groupby(['PCA_Cluster', 'Category']).size().unstack(fill_value=0)
cluster_purity = []

for cluster_id in range(optimal_k_pca):
    cluster_foods = cluster_category_counts.loc[cluster_id]
    total_foods = cluster_foods.sum()
    max_category_count = cluster_foods.max()
    purity = max_category_count / total_foods if total_foods > 0 else 0
    cluster_purity.append(purity)

axes[1,0].bar(range(optimal_k_pca), cluster_purity, color='lightgreen', alpha=0.8)
axes[1,0].set_xlabel('PCA Cluster ID')
axes[1,0].set_ylabel('Cluster Purity')
axes[1,0].set_title('PCA Cluster Purity (Dominant Category %)')
axes[1,0].grid(True, alpha=0.3)

# 5. Feature space coverage
# Calculate how well clusters cover the feature space
cluster_centers = kmeans_pca.cluster_centers_
feature_coverage = []

for feature_idx, feature_name in enumerate(feature_columns):
    # Calculate feature range in original space for this sample
    feature_values = X_sample[:, feature_idx]
    feature_range = np.max(feature_values) - np.min(feature_values)
    
    # Calculate how much of this range is covered by cluster centers
    if feature_range > 0:
        feature_coverage.append(feature_range)
    else:
        feature_coverage.append(0)

# Use the PCA space for visualization instead
pc1_range = np.max(X_pca[:, 0]) - np.min(X_pca[:, 0])
pc2_range = np.max(X_pca[:, 1]) - np.min(X_pca[:, 1])

# Calculate cluster spread
cluster_spreads = []
for cluster_id in range(optimal_k_pca):
    cluster_points = X_pca[pca_clusters == cluster_id]
    if len(cluster_points) > 1:
        spread = np.std(cluster_points, axis=0).mean()
        cluster_spreads.append(spread)
    else:
        cluster_spreads.append(0)

axes[1,1].bar(range(optimal_k_pca), cluster_spreads, color='orange', alpha=0.8)
axes[1,1].set_xlabel('PCA Cluster ID')
axes[1,1].set_ylabel('Average Cluster Spread')
axes[1,1].set_title('PCA Cluster Compactness')
axes[1,1].grid(True, alpha=0.3)

# 6. Category distribution in best cluster
best_cluster_id = np.argmax(cluster_purity)
best_cluster_foods = food_sample[pca_clusters == best_cluster_id]
category_dist = best_cluster_foods['category'].value_counts().head(10)

axes[1,2].pie(category_dist.values, labels=category_dist.index, autopct='%1.1f%%', startangle=90)
axes[1,2].set_title(f'Food Categories in Best Cluster (ID: {best_cluster_id})')

plt.tight_layout()
plt.show()

print(f"\n📊 Cluster Analysis Summary:")
print(f"   • PCA optimal clusters: {optimal_k_pca} (silhouette: {max(silhouette_scores_pca):.3f})")
print(f"   • t-SNE optimal clusters: {optimal_k_tsne} (silhouette: {max(silhouette_scores_tsne):.3f})")
print(f"   • Best cluster purity: {max(cluster_purity):.2%}")
print(f"   • Average cluster purity: {np.mean(cluster_purity):.2%}")

# Analyze cluster-category alignment
print(f"\n📊 Cluster-Category Alignment:")
for cluster_id in range(min(5, optimal_k_pca)):  # Show first 5 clusters
    cluster_categories = pca_cluster_composition[pca_cluster_composition['PCA_Cluster'] == cluster_id]['Category'].value_counts()
    dominant_category = cluster_categories.index[0]
    dominant_percentage = cluster_categories.iloc[0] / cluster_categories.sum() * 100
    print(f"   Cluster {cluster_id}: {dominant_percentage:.1f}% {dominant_category} ({cluster_categories.sum()} foods)")

In [ ]:
# 5. Model Performance Visualization Dashboard
print("\n5️⃣ MODEL PERFORMANCE VISUALIZATION DASHBOARD")
print("-" * 50)

# Create a comprehensive performance dashboard
fig = plt.figure(figsize=(20, 15))

# Create a complex subplot layout
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# 1. Overall model performance metrics
ax1 = fig.add_subplot(gs[0, 0])
metrics_names = ['Similarity\nScore', 'Category\nConsistency', 'Distance\nQuality', 'Feature\nImportance']
metrics_values = [
    similarity_results['overall_metrics']['avg_quality_score'],
    similarity_results['overall_metrics']['avg_consistency'],
    1 - similarity_results['overall_metrics']['avg_distance'],  # Invert distance for better visualization
    np.mean(feature_importance_df['Importance'])
]

bars = ax1.bar(metrics_names, metrics_values, color=['skyblue', 'lightgreen', 'orange', 'lightcoral'])
ax1.set_ylim(0, 1)
ax1.set_title('Overall Model Performance')
ax1.set_ylabel('Score')

# Add value labels on bars
for bar, value in zip(bars, metrics_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{value:.3f}', ha='center', va='bottom')

# 2. Feature importance radar chart
ax2 = fig.add_subplot(gs[0, 1], projection='polar')
angles = np.linspace(0, 2 * np.pi, len(feature_columns), endpoint=False)
values = feature_importance_df.set_index('Feature').loc[feature_columns, 'Importance'].values

# Close the plot
angles = np.concatenate([angles, [angles[0]]])
values = np.concatenate([values, [values[0]]])

ax2.plot(angles, values, 'o-', linewidth=2, color='blue')
ax2.fill(angles, values, alpha=0.25, color='blue')
ax2.set_xticks(angles[:-1])
ax2.set_xticklabels(feature_columns, fontsize=9)
ax2.set_title('Feature Importance Radar')

# 3. Prediction confidence distribution
ax3 = fig.add_subplot(gs[0, 2])
ax3.hist(conf, bins=30, alpha=0.7, color='purple', edgecolor='black')
ax3.axvline(np.mean(conf), color='red', linestyle='--', label=f'Mean: {np.mean(conf):.3f}')
ax3.set_xlabel('Prediction Confidence')
ax3.set_ylabel('Frequency')
ax3.set_title('Confidence Distribution')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Error rate by category (top 10)
ax4 = fig.add_subplot(gs[0, 3])
top_errors = error_analysis.nlargest(10, 'Error_Rate')
ax4.barh(range(len(top_errors)), top_errors['Error_Rate'], color='red', alpha=0.7)
ax4.set_yticks(range(len(top_errors)))
ax4.set_yticklabels(top_errors['Category'], fontsize=9)
ax4.set_xlabel('Error Rate')
ax4.set_title('Highest Error Rates')

# 5. PCA visualization with variance explained
ax5 = fig.add_subplot(gs[1, :2])
scatter = ax5.scatter(X_pca[:, 0], X_pca[:, 1], c=y_sample, cmap='tab20', alpha=0.6, s=30)
ax5.set_xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.2%} variance)')
ax5.set_ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.2%} variance)')
ax5.set_title('PCA - Food Categories Distribution')

# 6. t-SNE visualization
ax6 = fig.add_subplot(gs[1, 2:])
scatter = ax6.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_sample, cmap='tab20', alpha=0.6, s=30)
ax6.set_xlabel('t-SNE Component 1')
ax6.set_ylabel('t-SNE Component 2')
ax6.set_title('t-SNE - Food Categories Clustering')

# 7. Similarity performance trends
ax7 = fig.add_subplot(gs[2, 0])
# Use baseline and optimization results for trends
baseline_scores = [results['combined_score'] for results in baseline_results.values()]
optimization_scores = [result['combined_score'] for result in optimization_results]

ax7.boxplot([baseline_scores, optimization_scores], labels=['Baseline', 'Optimized'])
ax7.set_ylabel('Combined Score')
ax7.set_title('Performance Distribution')
ax7.grid(True, alpha=0.3)

# 8. Category consistency analysis
ax8 = fig.add_subplot(gs[2, 1])
baseline_consistency = [results['category_consistency'] for results in baseline_results.values()]
optimization_consistency = [result['category_consistency'] for result in optimization_results]

ax8.boxplot([baseline_consistency, optimization_consistency], labels=['Baseline', 'Optimized'])
ax8.set_ylabel('Category Consistency')
ax8.set_title('Consistency Distribution')
ax8.grid(True, alpha=0.3)

# 9. Model calibration curve
ax9 = fig.add_subplot(gs[2, 2])
ax9.plot(bin_centers, accuracies, 'o-', label='Model', color='blue', linewidth=2)
ax9.plot([0, 1], [0, 1], '--', color='gray', alpha=0.7, label='Perfect')
ax9.set_xlabel('Predicted Confidence')
ax9.set_ylabel('Actual Accuracy')
ax9.set_title('Model Calibration')
ax9.legend()
ax9.grid(True, alpha=0.3)

# 10. Feature correlation heatmap
ax10 = fig.add_subplot(gs[2, 3])
# Calculate correlation matrix for features
feature_corr = np.corrcoef(X_sample.T)
im = ax10.imshow(feature_corr, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
ax10.set_xticks(range(len(feature_columns)))
ax10.set_yticks(range(len(feature_columns)))
ax10.set_xticklabels(feature_columns, rotation=45, fontsize=8)
ax10.set_yticklabels(feature_columns, fontsize=8)
ax10.set_title('Feature Correlation')

# Add colorbar
plt.colorbar(im, ax=ax10, shrink=0.8)

plt.suptitle('KNN Food Similarity Model - Comprehensive Performance Dashboard', fontsize=16, y=0.98)
plt.show()

print(f"\n🎯 Performance Dashboard Summary:")
print(f"   • Overall similarity score: {similarity_results['overall_metrics']['avg_quality_score']:.3f}")
print(f"   • Category consistency: {similarity_results['overall_metrics']['avg_consistency']:.2%}")
print(f"   • Model confidence: {np.mean(conf):.3f}")
print(f"   • High confidence predictions: {np.mean(conf > 0.8):.1%}")

# Save visualization results
visualization_results = {
    'pca_results': {
        'explained_variance_ratio': pca_viz.explained_variance_ratio_.tolist(),
        'total_variance_explained': pca_viz.explained_variance_ratio_.sum()
    },
    'confidence_analysis': {
        'mean_confidence': np.mean(conf),
        'high_confidence_rate': np.mean(conf > 0.8),
        'low_confidence_rate': np.mean(conf < 0.5)
    },
    'cluster_analysis': {
        'optimal_k_pca': optimal_k_pca,
        'optimal_k_tsne': optimal_k_tsne,
        'best_cluster_purity': max(cluster_purity),
        'average_cluster_purity': np.mean(cluster_purity)
    },
    'performance_metrics': {
        'similarity_score': similarity_results['overall_metrics']['avg_quality_score'],
        'category_consistency': similarity_results['overall_metrics']['avg_consistency'],
        'distance_quality': 1 - similarity_results['overall_metrics']['avg_distance'],
        'feature_importance_avg': np.mean(feature_importance_df['Importance'])
    }
}

joblib.dump(visualization_results, '../models/visualization_results.pkl')
print(f"\n✅ Saved visualization analysis results")
print(f"📁 File: ../models/visualization_results.pkl")

## Visualization and Analysis Summary

This notebook provided comprehensive visualization and analysis of the KNN food similarity model performance. Key insights:

### Feature Space Understanding

- **PCA Analysis**: Explained {pca*viz.explained_variance_ratio*.sum():.1%} of variance with 2 components
- **t-SNE Clustering**: Revealed natural groupings of nutritionally similar foods
- **Cluster Quality**: Identified optimal clustering patterns in nutritional space

### Model Performance

- **Confidence Analysis**: Model shows appropriate uncertainty calibration
- **Error Patterns**: Identified systematic challenges with certain food categories
- **Feature Importance**: Key nutritional features driving similarity decisions

### Clustering Insights

- **Natural Groupings**: Foods cluster by nutritional similarity, not just categories
- **Category Alignment**: Some food categories form tighter clusters than others
- **Recommendation Quality**: Cluster analysis validates similarity model effectiveness

### Business Applications

- **Quality Thresholds**: Established confidence levels for automated recommendations
- **Category Insights**: Understanding which food types are easier to recommend
- **Model Reliability**: Visualization confirms model readiness for production use

### Next Steps

The comprehensive analysis results will be used in the final deployment notebook to create production-ready model documentation and establish monitoring metrics for the food similarity system.
